# Tutorial 4: Eco-Geomorphological simulation with Bathtub model
The Bathtub model/method/approach is a simplistic representation of the water and sediment dynamic over the floodplain. It assumes that water flows over the floodplain without attenuation effects due to surface roughness or any sort of obstruction. Similarly, the sediment concentration is computed by a linear relationship between the maximum depth and the concentration in the source water body. On the other hand, this model is extremely fast, making it a good start for any EGM modelling project.


In this tutorial, we will see how to run an Eco-Geomorphological (EGM) simulation using the classes/functions provided in the **egmlib**. As the *Bathtub* model if not affected by the domain parameters/structures, we will use the simple *single row only*  domain created in the `example_domain2` notebook.


## Dependencies
* You will need all files created with `example_domain2`.
* CSV files with water levels and sediment concentration input series generated with `Tutorial 1 - Creating a new domain`

***
# STEP 0: Preparing files and loading the egmlib library
Create a folder named **tutorial04** and copy all files from **domain02** to this new folder.


In the code below replace the string `lib_folder` with the path to the folder where the **egmlib.py** file is and then run the cell. Just note that we should use `/` or `\\` instead of a single `\` as Python understand this last as an indicator of a special character.


The egmlib library already includes the import of many other libraries, as **numpy** and **matplotlib** that will be used later in this notebook.

In [ ]:
lib_folder = 'C:/Wetlands/Tutorials'
from sys import path
path.append(lib_folder)
from egmlib import *

# STEP 1: Initialising the EGM Framework
As we saw in tutorials 2 and 3, to initialise an instance of **EGMframework** class we need to provide a file with the simulation settings and the directory's path with all files to run the Hydrodynamic model.



## 1.1 Settings file
As now we will use all EGM models from the library, the settings file will have quite more information. In this tutorial we will create both a `json` and a `csv` file with the same content, just to practice, as the *Framework* accepts both types.

Content to be written in the simulation setting's file:
- Simulation title (not useful at all, but good to use as reference)
- Time-step (years) between stages of the EGM simulation
- Initial and Final EGM stages
- Vegetation's roughness coefficient table
- Hillslope diffusion coefficient
- Path to the file with input water level series
- Path to the file with input sediment concentration series
- Boolean to tell if the input files have the first column as index/time-labels


Back to Tutorial 1, we've created an input series of water levels using the sinusoidal function and constant sediment. Using the RCPP 8.5 scenario, we add the expected SLR to the initial series to represent the water levels from 2000 to 2100 every two decades. Each series is 4-days-long with records every 5 minutes (300 s).


The vegetation roughness coefficients are the same used in other studies for the Hunter Estuary (SE Australia) area. The vegetation codes are as follow:
- 0 = no-vegetation
- 1 = freshwater vegetation
- 2 = saltmarsh
- 3 = mangrove

In [ ]:
# Directory's path where the model will run
run_folder = 'C:\\Wetlands\\Tutorials\\tutorial04'

# Creating dictionary with information for EGM simulation
settings = {
    'title': 'Bathtub',
    'Dt_EGM': 20,
    'start':'2000',
    'end':'2100',
    'roughness': {
        0: 0.12,
        1: 0.12,
        2: 0.15,
        3: 0.50
    },
    'Dcoef': 8.0,
    'input_levels_file': 'C:\\Wetlands\\Tutorials\\water_levels_1.csv',
    'input_sediment_file': 'C:\\Wetlands\\Tutorials\\sediment_conc_1.csv',
    'first_column_as_index': 'yes'
}

# Saving settings in a json file
filename = os.path.join(run_folder, 'egm_setup.json')
with open(filename, 'w') as file:
    json.dump(settings, file)

# Now let's create the equivalent CSV file with the same settings. As the CSV is a text file, we can't specify
#the type of each information nor group multilevel information (as the roughness coefficient for each vegetation
#code). However, as the key-entries must follow a given name, the EGMFramework knows how to deal with each piece.
filename = os.path.join(run_folder, 'egm_setup.csv')
file = open(filename, 'w')
file.write('''title,Tutorial 4
Dt_EGM,20
start,2000
end,2100
roughness,0,0.12
roughness,1,0.12
roughness,2,0.15
roughness,3,0.50
Dcoef,8.0
input_levels_file,C:\\Wetlands\\Tutorials\\water_levels_1.csv
input_sediment_file,C:\\Wetlands\\Tutorials\\sediment_conc_1.csv
first_column_as_index,yes''')
file.close()

## 1.2 Preparing an EGMframework instance
Create an instance from EGMframework class and then call its method **initialize(** *settings_file*, *run_folder* **)**

In [ ]:
# Creating a new instance of EGMframework
egm = EGMframework()
egm.initialize(filename, run_folder)

# Printing some attributes of egm
print('\n\nNumber of cells and links:', egm.NoC, egm.NoL)
print('Water depths boundary condition applied to:', egm.loc_HT)
print('Series for EGM simulation:')
for i in range(egm.Nstages):
    print('    series %i: %s' % (i, egm.stages[i]))
print('Time step between stages: %i years' % egm.Dt_EGM)
print('Rule for EGM models: %s' % egm.rule)    #should return the default option: 'SE Australia'
periods_write(egm.periods, 'periods.csv', stepNames=egm.stages)

# STEP 2: Running the EGM Simulation with Bathtub approach
To run the EGM simulation we must call the models following a sequence that, if model "A" needs the outputs of model "B", then "B" must be ran before "A". Some special steps might be done at the begining/end of each EGM stage, to ensure a continuity of the simulation.


The EGM simulation evolves through *stages* (see definition below), and the outputs are stored in matrices already created as attributes of the EGMframework instance. In the settings file we defined a *start* and an *end* stage, which in this case match with the head label of the first and last columns in the input levels CSV file. Actually, this is the default behaviour when *start* and *end* are not provided. Thus, these entries are more useful when we want to run an interval in between the first and last stage provided in the input file.


The time between stages must be informed if a model that returns a value/year will be used (like the *Accretion model*). We did that by providing the entry *Dt_EGM* as 20 years. The *Hillslope Diffusion model*  also require this information as the *Dcoef* (diffusivity coefficient) is in m$^2$/year. 


When the simulation is concluded, we call the method *finalize()* to save the EGMframework instance in a binary file. The **egmlib** has a few functions to read this file and reshape the output data in 3D matrices like: (number of EGM stages, number of rows, number of columns), which is easier to select specific rows/columns or areas of the domain when plotting.


***
***
A **stage** is a single step in the EGM simulation. It can be a single year like in a run from 2000 to 2100, or just a id/name. **Stages** are taken from the header (column labels) of input water levels CSV file, and follow the same order presented there (i.e. from left to right). Parallel to stages we have the **stage index (stix)**, which is a simple numeration of these stages (first stage = 0, second stage = 1, and so on). This one is necessary to access those datasets that are not pandas.DataFrame, like the EGM outputs which are numpy.ndarrays. 
***
***

In [ ]:
t0 = datetime.now()
for stix, stage in enumerate(egm.stages):
    
    print ('\nProcessing stage:', stage)
    
    if stix > 0:
        #Update parameters
        egm.update_parameters(stix-1)
        
        #For the bathtub simulation we don't need to update the parameter's file or domain's file, as we won't
        #run the Hydrodynamic model...
        #domain_update(z=egm.z, chndep=egm.chndep)
        #parameters_update(params=egm.params)
    
    #Run the Bathtub model to obtain water depths series, return outcomes as a highFreqData instance
    hfd = egm.update_bathtub(stage, returnType='highFreqData')
    
    #Compute Hydroperiod and Mean-Depth-Below-High-Tide
    egm.H[stix,:], egm.D[stix,:] = egm.update_tidal_indexes(stix, hfd.h)
    
    #Define vegetation cover
    egm.V[stix,:] = egm.update_vegetation(stix)
    
    #Compute sediment concentration (using the linear relationship)
    egm.C[stix,:] = egm.update_linear_sediment(stix)
    
    #Compute accretion (already scaled to self.Dt_EGM years) and add it to the surface
    egm.A[stix,:] = egm.update_accretion(stix)
    egm.z, egm.chndep = add_accretion(egm.z, egm.chndep, egm.A[stix,:])
    
    #Apply Hillslope Diffusion model (egm.EG = elevation gain ==> egm.z(now) - egm.z(at start))ls
    egm.EG[stix,:] = egm.update_diffusion()
    
    #Save the current stage index in an attribute to pick from this point on if the simulation crashes
    egm.current = stix

print('\nElapsed time:', (datetime.now() - t0))
egm.finalize()    #'.finalize()' creates a pickle file. But '.finalize("filename.csv")' creates a CSV one

# STEP 3: Results visualization
Once the EGM simulation is done, we can load the results using the function **load_to_plot(** *results_file*, *apply_exclusion* **)**. This function not only loads the datasets created in the EGM simulation, but also computes the bottom elevation every stage, the accumulated accretion, and the meshgrid for vegetation plots. Another important operation is reshaping the EGM-data matrices from (Nstages, NoC) to (Nstages, Nrows, Ncols), which makes easier to select specific rows/columns of the domain. Moreover, if the second argument is True, all cells from special areas will have their values turned to NaN, thus won't be shown in any plot.


What we will do now is a set of 3 plots for each stage of the EGM Simulation. The first plot will show the Hydroperiod and the Mean Depth Below High Tide. The second plot will show Accumulated Accretion and Elevation Gain. The last plot will show a vegetation map.


***
If the visualization area become an up/down scrolling window, you may use the shorcut: SHIFT+o to toggle scrolling
***

In [ ]:
#Load results saved with egm.finalize() in the block above
data = load_to_plot('framework_data.pickle', True)

#Plot results for each stage (select this cell and press 'o' after run it to un-colapse the visualization area)
for i in range(data['shape'][0]):
    
    fig, (ax1, ax3, ax5) = pyplot.subplots(nrows=3, dpi=100, figsize=(6,6), constrained_layout=True)
    
    #Hydroperiod
    p1, = ax1.plot(data['xc'][0,:], data['H'][i,0,:], label='Hydroperiod', color='tab:brown')
    ax1.set_ylabel('Hydroperiod (%)', color='tab:brown')
    ax1.tick_params(axis='y', labelcolor='tab:brown')
    ax1.set_ylim(np.nanmin(data['H']) - 1, np.nanmax(data['H']))
    ax1.set_xlim(0,1000)
    ax1.grid()
    
    #Mean Depth Below High Tide
    ax2 = ax1.twinx()
    p2, = ax2.plot(data['xc'][0,:], data['D'][i,0,:], label='Depth High Tide', color='tab:blue')
    ax2.set_ylabel('Mean Depth Below\nHigh Tide (m)', color='tab:blue')
    ax2.tick_params(axis='y', labelcolor='tab:blue')
    ax2.set_ylim(np.nanmin(data['D'])-0.01, np.nanmax(data['D']))
    
    #Legend first plot
    lines = [p1, p2]
    ax1.legend(lines, [l.get_label() for l in lines])
    
    #Elevation Gain
    p3, = ax3.plot(data['xc'][0,:], data['EG'][i,0,:], label='Elevation Gain', color='tab:olive')
    ax3.set_ylabel('Elevation Gain (m)', color='tab:olive')
    ax3.tick_params(axis='y', labelcolor='tab:olive')
    ax3.set_ylim(np.nanmin(data['EG'])-0.01, np.nanmax(data['EG']))
    ax3.set_xlim(0,1000)
    ax3.grid()
    
    #Accumulated Accretion
    ax4 = ax3.twinx()
    p4, = ax4.plot(data['xc'][0,:], data['acA'][i,0,:], label='Accumulated Accretion', color='tab:cyan')
    ax4.set_ylabel('Accumulated Accretion (m)', color='tab:cyan')
    ax4.tick_params(axis='y', labelcolor='tab:cyan')
    ax4.set_ylim(np.nanmin(data['acA'])-0.01, np.nanmax(data['acA']))
    
    #Legend second plot
    lines = [p3, p4]
    ax3.legend(lines, [l.get_label() for l in lines])
    
    #Vegetation map
    #the vegetation map is usually a bit complex to be made, on the other hand we rarely do it in a different way
    #thus, let's use the plot_vegetation() function that will do the default vegetation map plotting for us
    plot_vegetation(axis=ax5, xgrid=data['xf'], ygrid=data['yf'], veg=data['V'][i,:,:], rule=data['rule'])
    
    fig.suptitle(data['stages'][i])
    pyplot.show()
    pyplot.close(fig)

#Press SHIFT+o to toggle scrolling of the output area

# The End
In this quick tutorial you saw how to prepare the settings file for an EGM simulation, run a Bathtub scenario and make a few plots from the results.


In the next tutorial we will run a similar case, but using the Hydrodynamic/Sediment Transport (HST) models. We will also create a few more plots to compare the new results with those produced here